In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(4)

## Load Model

In [4]:
name = '04_xse_resnext_512_appian_alb'

In [5]:
import pretrainedmodels

In [6]:
m = pretrainedmodels.se_resnext50_32x4d(num_classes=6, pretrained=None)

In [7]:
m.avg_pool = nn.AdaptiveAvgPool2d(1)

In [8]:
# state['model'].keys()

In [9]:
# sp2 = '~/kaggle/rsna_retro/models/model100/fold0_ep2.pt'
sp3 = '~/kaggle/rsna_retro/models/model100/fold0_ep3.pt'

saved_path = Path(sp3).expanduser()
state = torch.load(saved_path, map_location='cpu')
m.load_state_dict(state['model'])

<All keys matched successfully>

In [10]:
dls = get_album_data(64, 384, splits=Meta.splits_stg1, img_dir=path_jpg)
learn = get_learner(dls, m)

## Training

In [11]:
do_fit(learn, 3, 5e-4, freeze=True)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.071588,0.079748,0.975101,0.951047,1:53:14


epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.072019,0.076472,0.976439,0.953352,1:53:42
1,0.063119,0.071228,0.977949,0.956904,1:54:25
2,0.056388,0.069510,0.978690,0.958304,1:53:33


## Submission

In [11]:
learn.load(f'runs/{name}-1')
sub_fn = f'subm/{name}'

In [12]:
learn.dls = get_test_data(Meta.df_tst, bs=128, sz=384, tst_dir='tst_jpg')

In [13]:
ob = learn.dls.one_batch()

In [14]:
preds,targs = learn.get_preds()

In [15]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [16]:
api.competition_submit(f'{sub_fn}.csv', f'{name} 384 384 384 appian se_resnext50', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 11.9MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [17]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14453417,
 'totalBytes': 27277209,
 'date': '2020-02-11T18:51:32.45Z',
 'description': '04_xse_resnext_512_appian_alb 384 384 384 appian se_resnext50',
 'errorDescription': None,
 'fileName': '04_xse_resnext_512_appian_alb.csv',
 'publicScore': None,
 'privateScore': None,
 'status': 'pending',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14453417/14453417.raw'}

## Save Predictions

In [11]:
path_appfeat512 = path/'appian_features_384_album'
path_appfeat512_tst = path/'appian_tst_features_384_album'

In [12]:
fn_save = f'runs/{name}-1'

In [13]:
# dls = get_test_data(Meta.df_tst, bs=256, sz=None, tst_dir='tst_jpg')
learn.dls = get_test_data(Meta.df_tst, bs=128, sz=384, tst_dir='tst_jpg')
learn.load(fn_save)
learn.model.last_linear = nn.Identity()
save_features(learn, path_appfeat512_tst)

In [18]:
tst_pth = path_appfeat512_tst.ls()[0]

In [14]:
learn.dls = get_test_data(Meta.df_comb, bs=128, sz=None, tst_dir='nocrop_jpg')
save_features(learn, path_appfeat512)